# SDSS dr13 Skyserver SQL Crawler
## SQL examples
```SQL
-- This query does a table JOIN between the imaging (PhotoObj) and spectra
-- (SpecObj) table includes the necessary columns in the SELECT to upload
-- (dbo.fGetNearbyObjEq) function finds nearby object ra-dec coordinates within [ra - theta, ra + theta] and [dec - theta, dec + theta]
-- magnitude = petroMag - extinction for each ugriz filters
-- 


SELECT
   p.objid, p.ra, p.dec, p.l, p.b,
   p.petroMag_u-p.extinction_u AS mag_u,
   p.petroMag_g-p.extinction_g AS mag_g,
   p.petroMag_r-p.extinction_r AS mag_r,
   p.petroMag_i-p.extinction_i AS mag_i,
   p.petroMag_z-p.extinction_z AS mag_z,
   s.z as redshift, s.zErr as redshift_err,
   s.velDisp, s.velDispErr,
   passive.logmass as passive_logmass,
   ROUND(pca.mstellar_median, 2) as pca_logmass,
   ROUND(passive.logmass-passive.minlogmass, 3) as logmass_lowerErr,
   ROUND(passive.maxlogmass-passive.logmass, 3) as logmass_upperErr,
   passive.absMagK,
   pca.vdisp_median as pca_vdisp, pca.vdisp_err as pca_vdisp_err,
   zoo.p_el_debiased as frac_ellip,
   zoo.p_cs_debiased as frac_spiral
FROM Galaxy AS p
   JOIN dbo.fGetNearbyObjEq({ra}, {dec}, {theta}) AS GN ON GN.objid = p.objid
   JOIN SpecObj AS s ON s.bestobjid = p.objid
   LEFT JOIN stellarMassPassivePort AS passive ON passive.specobjid = s.specobjid
   LEFT JOIN stellarMassPCAWiscM11 AS pca ON pca.specobjid = s.specobjid
   LEFT JOIN zooSpec AS zoo ON zoo.specobjid = s.specobjid
WHERE
   s.class = 'GALAXY'
   AND s.z BETWEEN {z_lower} AND {z_upper}
```

### Sitemap
- [SDSS dr13 Skyserver Schema Browser](http://skyserver.sdss.org/dr13/en/help/browser/browser.aspx)
- [SQL Search](http://skyserver.sdss.org/dr13/en/tools/search/sql.aspx)

## Crawling data description

|Column|Description|Unit|
|------|-----------|----|
|objid|Object number in SDSS|-|
|ra|Right Ascension|deg|
|dec|Declination|deg|
|l|galactic longitude|deg|
|b|galactic latitude|deg|
|mag_u|Petrosian u-band magnitude corrected for extinction|mag|
|mag_g|Petrosian g-band magnitude corrected for extinction|mag|
|mag_r|Petrosian r-band magnitude corrected for extinction|mag|
|mag_i|Petrosian i-band magnitude corrected for extinction|mag|
|mag_z|Petrosian z-band magnitude corrected for extinction|mag|
|z|Redshift corresponds to no-QSO redshift|-|
|zErr|Redshift error|-|
|velDisp|Velocity dispersion|km/s|
|velDispErr|Velocity dispersion error|km/s|
|passive_logmass|$logM$|$logM_{\odot}$|
|pca_logmass|$logM$ measured by pca method|$logM_{\odot}$|
|logmass_lowerErr|pca_logmass lower error|$logM_{\odot}$|
|logmass_upperErr|pca_logmass upper error|$logM_{\odot}$|
|absMagK|Absolute magnitude in K inferred from model|mag|
|pca_vdisp|PCA Velocity dispersion|km/s|
|pca_vdisp_err|PCA Velocity dispersion Error|km/s|
|frac_ellip|debiased fraction of votes for elliptical|-|
|frac_spiral|debiased fraction of votes for spiral|-|

## GalWCat19 cluster data description

|Column|Description|Unit|
|------|-----------|----|
|  Ng   |Number of total galaxy members for this cluster|-|
| ClID  |           Cluster Identifier        |-|
|RAJ2000|Cluster center right ascension(J2000)|deg|
|DEJ2000|Cluster center right ascension(J2000)|deg|
|   z   |Redshift corresponds to no-QSO redshift|-|
|   RV  |      Cluster radial velocity        |$km/s$|
|CoDist |          Comoving distance          |$h^{-1}Mpc$|
|  r500 |Radius from the cluster center at which $\rho=\Delta_{500}\rho_{c}$|$h^{-1}Mpc$|
|  N500 |Number of all member galaxies within r500|-|
|sig500 |Velocity dispersion within r500|km/s|
| M500  |Cluster mass within r500|$M_{\odot}$|
| r200  |Radius from the cluster center at which $\rho=\Delta_{200}\rho_{c}$|$h^{-1}Mpc$|
| N200  |Number of all member galaxies within r200|-|
| sig200|Velocity dispersion within r200|km/s|
| M200  |Cluster mass within r200|$M_{\odot}$|
| r100  |Radius from the cluster center at which $\rho=\Delta_{100}\rho_{c}$|$h^{-1}Mpc$|
| N100  |Number of all member galaxies within r100|-|
| sig100|Velocity dispersion within r100|km/s|
| M100  |Cluster mass within r100|$M_{\odot}$|
| r005  |Radius from the cluster center at which $\rho=\Delta_{005}\rho_{c}$|$h^{-1}Mpc$|
| N005  |Number of all member galaxies within r005|-|
| sig005|   Velocity dispersion within r005  |km/s|
| M005  |      Cluster mass within r005      |$M_{\odot}$|
|  Rs   |           NFW scale radius         |$h^{-1}Mpc$|
|  Ms   |            NFW scale mass          |$M_{\odot}$|
| conc  |    Concentration = $R_{200}/R_s$   |-|

## GalWCat19 galaxy members data description

|Column|Description|Unit|
|------|-----------|----|
|RAJ2000|Cluster center right ascension(J2000)|deg|
|DEJ2000|Cluster center right ascension(J2000)|deg|
|z|Redshift corresponds to no-QSO redshift|-|
|ClID|Cluster Identifier|-|

In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import scipy.integrate as integrate

import warnings
warnings.filterwarnings(action='ignore')

import time
import math
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os

c = 299792.458
sign = 8
multiple = 6
rad2deg = 180 / np.pi

In [28]:
def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('headless')
    driver = webdriver.Chrome(executable_path="../chromedriver",
                              options=chrome_options)
    return driver

In [31]:
def make_query(ra, dec, theta, r200, dist, z_upper, z_lower):
    query = f"""SELECT
   p.objid, p.ra, p.dec, p.l, p.b,
   p.petroMag_u-p.extinction_u AS mag_u,
   p.petroMag_g-p.extinction_g AS mag_g,
   p.petroMag_r-p.extinction_r AS mag_r,
   p.petroMag_i-p.extinction_i AS mag_i,
   p.petroMag_z-p.extinction_z AS mag_z,
   s.z, s.zErr, s.velDisp, s.velDispErr,
   passive.logmass as passive_logmass,
   ROUND(pca.mstellar_median, 2) as pca_logmass,
   ROUND(passive.logmass-passive.minlogmass, 3) as logmass_lowerErr,
   ROUND(passive.maxlogmass-passive.logmass, 3) as logmass_upperErr,
   passive.absMagK,
   pca.vdisp_median as pca_vdisp, pca.vdisp_err as pca_vdisp_err,
   zoo.p_el_debiased as frac_ellip,
   zoo.p_cs_debiased as frac_spiral
FROM Galaxy AS p
   JOIN dbo.fGetNearbyObjEq({ra}, {dec}, {theta}) AS GN ON GN.objid = p.objid
   JOIN SpecObj AS s ON s.bestobjid = p.objid
   LEFT JOIN stellarMassPassivePort AS passive ON passive.specobjid = s.specobjid
   LEFT JOIN stellarMassPCAWiscM11 AS pca ON pca.specobjid = s.specobjid
   LEFT JOIN zooSpec AS zoo ON zoo.specobjid = s.specobjid
WHERE
   s.class = 'GALAXY'
   AND s.z BETWEEN {z_lower} AND {z_upper}"""
    return query

In [33]:
def get_csv(driver, query):
    driver.find_element(By.XPATH, '//*[@id="clear"]').click()
    driver.find_element(By.XPATH, '//*[@id="cmd"]').send_keys(query)
    driver.find_element(By.XPATH, '//*[@id="transp"]/form/table/tbody/tr[3]/td/table/tbody/tr/td[2]/table/tbody/tr[1]/td[4]/input').click()
    driver.find_element(By.XPATH, '/html/body/div[4]/form/table/tbody/tr[3]/td/table/tbody/tr/td[1]/table/tbody/tr[6]/td/input').click()
    
    time.sleep(np.random.randint(20,30))

In [7]:
def digitScaler(num, sign):
    digit = math.floor(np.log10(num))
    n = num / 10**digit
    n = round(n, sign-1)
    return n * math.pow(10,digit)

def redshift(v_r):
    # calculating relativistic redshift with radial velocity
    return np.sqrt((c + v_r)/(c - v_r)) - 1

def radial_v(z):
    # calculating relativistic radial velocity with redshift
    return c * ((z+1)**2 - 1)/((z+1)**2 + 1)

def dimless_h(z, h_0=1):
    # Calculating dimesionless hubble constant, h(z)
    # According to the GalWCat19 paper,
    # Omega(r, m, k, gamma) = (0, 0.3, 0, 0.7)
    # h_0 = 0.7
    return h_0 * np.sqrt(0.3*(1+z)**3 + 0.7)

def comv_dist(z):
    return (c / 100) * integrate.quad(lambda x: 1/dimless_h(x), 0, z)[0]

def angular_dist(delta_ra, delta_dec):
    tr = np.pi / 180
    return np.arccos(np.cos(delta_ra * tr) * np.cos(delta_dec * tr)) / tr

In [9]:
header = ['ra', 'dec', 'z', 'v_r',
          'comv_dist', 'r_200', 'N_200', 'sigma_200',
          'sigma_200_lowerErr', 'sigma_200_upperErr', 'M_200', 'M_200_Err',
          'R_s', 'R_s_Err', 'M_s', 'M_s_Err']
cls_info = pd.read_csv('./data/GalWCls19_summary.csv',index_col=0)
cls_info.columns = header
cls_info

,ra,dec,z,v_r,comv_dist,r_200,N_200,sigma_200,sigma_200_lowerErr,sigma_200_upperErr,M_200,M_200_Err,R_s,R_s_Err,M_s,M_s_Err
ID,,,,,,,,,,,,,,,,
1,230.65973,27.743846,0.073177,21137.1930,215.714310,1.758845,167,1042.08600,-83.233978,100.849290,1.350000e+15,3.570000e+14,0.41,0.108,3.060000e+14,8.060000e+13
2,227.55402,33.498096,0.113908,32215.0730,332.547940,1.511349,63,926.75761,-92.042355,118.740250,8.950000e+14,2.350000e+14,0.32,0.084,1.880000e+14,4.930000e+13
3,194.93502,27.912461,0.023372,6924.9620,69.697942,1.544850,672,932.60305,-47.761507,55.312723,8.760000e+14,2.230000e+14,0.33,0.084,1.850000e+14,4.710000e+13
4,258.20851,64.052953,0.080997,23301.8680,238.327350,1.453410,155,881.24884,-70.222098,82.083225,7.700000e+14,2.080000e+14,0.37,0.100,1.860000e+14,5.040000e+13
5,209.83093,27.968947,0.075065,21661.5240,221.182270,1.448992,77,842.65332,-83.231049,96.160155,7.590000e+14,2.170000e+14,0.14,0.040,9.660000e+13,2.760000e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,124.92122,21.032812,0.093382,26693.3160,273.964500,0.689068,10,387.83579,-76.069234,103.545850,8.310000e+13,4.650000e+13,0.05,0.028,9.110000e+12,5.100000e+12
997,134.80258,39.671084,0.027620,8165.9917,82.285446,0.703441,12,449.75002,-251.333430,196.605620,8.300000e+13,3.320000e+13,0.04,0.016,8.110000e+12,3.240000e+12
998,164.50680,1.634841,0.039362,11568.5010,116.952150,0.700718,51,421.30468,-72.371673,106.786410,8.290000e+13,2.330000e+13,0.37,0.104,3.930000e+13,1.100000e+13


In [8]:
cls1 = cls_info.iloc[0,:]

v = cls1['v_r']
z_calc = redshift(v)
v_calc = radial_v(cls1['z'])
print(digitScaler(v_calc, sign))
print(v)
print(cls1['comv_dist'])
print(digitScaler(comv_dist(cls1['z']), sign))

21137.193
21137.193
215.71431
215.71430999999998


In [187]:
print(digitScaler(angular_dist(1, 1), sign_fig))

1.4141777
1.4142135623730951


In [22]:
idx = 16
ra = cls_info.loc[idx, 'ra']
dec = cls_info.loc[idx, 'dec']
dist = cls_info.loc[idx, 'comv_dist']
r200 = cls_info.loc[idx, 'r_200']
v_r = cls_info.loc[idx, 'v_rad']
sigma = cls_info.loc[idx, 'sigma_200']

z_upper = redshift(v_r + multiple * sigma)
z_lower = redshift(v_r - multiple * sigma)

query = make_query(ra, dec, r200, dist, z_upper, z_lower)
print(query)

SELECT
   p.objid, p.ra, p.dec, p.l, p.b,
   p.petroMag_u-p.extinction_u AS mag_u,
   p.petroMag_g-p.extinction_g AS mag_g,
   p.petroMag_r-p.extinction_r AS mag_r,
   p.petroMag_i-p.extinction_i AS mag_i,
   p.petroMag_z-p.extinction_z AS mag_z,
   s.z, s.zErr, s.velDisp, s.velDispErr,
   passive.logmass as passive_logmass,
   ROUND(pca.mstellar_median, 2) as pca_logmass,
   ROUND(passive.logmass-passive.minlogmass, 3) as logmass_lowerErr,
   ROUND(passive.maxlogmass-passive.logmass, 3) as logmass_upperErr,
   passive.absMagK,
   pca.vdisp_median as pca_vdisp, pca.vdisp_err as pca_vdisp_err,
   zoo.p_el_debiased as frac_ellip,
   zoo.p_cs_debiased as frac_spiral
FROM Galaxy AS p
   JOIN dbo.fGetNearbyObjEq(10.400877, -9.3250982, 1.6301228566004442) AS GN ON GN.objid = p.objid
   JOIN SpecObj AS s ON s.bestobjid = p.objid
   LEFT JOIN stellarMassPassivePort AS passive ON passive.specobjid = s.specobjid
   LEFT JOIN stellarMassPCAWiscM11 AS pca ON pca.specobjid = s.specobjid
   LEFT JOI

In [25]:
url = "http://skyserver.sdss.org/dr13/en/tools/search/sql.aspx"
driver = set_chrome_driver()
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="clear"]').click()
driver.find_element(By.XPATH, '//*[@id="cmd"]').send_keys(query)
driver.find_element(By.XPATH, '//*[@id="transp"]/form/table/tbody/tr[3]/td/table/tbody/tr/td[2]/table/tbody/tr[1]/td[4]/input').click()
driver.find_element(By.XPATH, '/html/body/div[4]/form/table/tbody/tr[3]/td/table/tbody/tr/td[1]/table/tbody/tr[6]/td/input').click()

time.sleep(3)
first_tab = driver.window_handles[0]
driver.switch_to.window(window_name=first_tab)

# for idx, cls1 in cls_info.iloc[1000:,:].iterrows():
#     ra = cls1['ra']
#     dec = cls1['dec']
#     dist = cls1['comv_dist']
#     r200 = cls1['r_200']
#     v_r = cls1['v_rad']
#     sigma = cls1['sigma_200']
    
#     z_upper = redshift(v_r + multiple * sigma)
#     z_lower = redshift(v_r - multiple * sigma)
    
#     query = make_query(ra, dec, r200, dist, z_upper, z_lower)
# #     get_html(driver, query)
#     get_csv(driver, query)
    
# #     if idx > 0:
# #         break

### solution method with delay issues

In [27]:
def get_missing_data(driver, cls_info, index, multiple):
    ra = cls1['ra']
    dec = cls1['dec']
    dist = cls1['comv_dist']
    r200 = cls1['r_200']
    v_r = cls1['v_rad']
    sigma = cls1['sigma_200']
    theta = multiple * np.arcsin(r200 / dist) * (180 / np.pi) * 60
    print('Index: {0} & process: {1:.2f}%'.format(idx, 100 * idx/N))
    
    z_upper = redshift(v_r + multiple * sigma)
    z_lower = redshift(v_r - multiple * sigma)
    
    query = make_query(ra, dec, theta, r200, dist, z_upper, z_lower)
    get_csv(driver, query)

In [49]:
get_missing_data(1201)

## GalWgal another properties crawler

In [41]:
header = ['N_gal', 'ra', 'dec',
          'z', 'v_rad', 'comv_dist',
          'r_500', 'N_500', 'sigma_500', 'M_500',
          'r_200', 'N_200', 'sigma_200', 'M_200',
          'r_100', 'N_100', 'sigma_100', 'M_100',
          'r_005', 'N_005', 'sigma_005', 'M_005',
          'R_s', 'M_s', 'concentration']

cls_info = pd.read_csv('./data/galwcls.csv', index_col=0)
cls_info = cls_info.reset_index()
cls_info = cls_info.drop(columns=['ClID'])
cls_info.index = cls_info.index + 1
cls_info.columns = header
cls_info.iloc[:1000, :]

,N_gal,ra,dec,z,v_rad,comv_dist,r_500,N_500,sigma_500,M_500,...,N_100,sigma_100,M_100,r_005,N_005,sigma_005,M_005,R_s,M_s,concentration
1,166,230.65973,27.743846,0.073177,21137.1930,215.714310,1.151608,123,1132.33920,9.506491e+14,...,191,991.86324,1.689758e+15,7.794941,446,706.55424,3.242931e+15,0.41,3.060947e+14,4.289865
2,61,227.55402,33.498096,0.113908,32215.0730,332.547940,0.995621,45,1007.64840,6.394636e+14,...,76,873.68274,1.104408e+15,6.618137,143,695.01505,2.066015e+15,0.32,1.879991e+14,4.722965
3,670,194.93502,27.912461,0.023372,6924.9620,69.697942,1.017135,485,992.56078,6.248669e+14,...,801,897.76898,1.081975e+15,6.772127,1389,750.50479,2.028706e+15,0.33,1.852209e+14,4.681364
4,149,258.20851,64.052953,0.080997,23301.8680,238.327350,0.945980,103,863.31423,5.309468e+14,...,183,838.62740,9.705232e+14,6.516373,314,683.99230,1.909044e+15,0.37,1.864585e+14,3.928135
5,75,209.83093,27.968947,0.075065,21661.5240,221.182270,0.990827,56,858.50327,6.065896e+14,...,91,817.41602,8.778047e+14,5.876928,189,632.52100,1.392339e+15,0.14,9.659857e+13,10.349945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,10,124.92122,21.032812,0.093382,26693.3160,273.964500,0.475609,10,387.83579,6.830086e+13,...,10,387.83579,9.451712e+13,2.738279,12,356.41517,1.433846e+14,0.05,9.112707e+12,13.781365
997,12,134.80258,39.671084,0.027620,8165.9917,82.285446,0.488771,8,537.94560,6.961029e+13,...,13,437.73512,9.329420e+13,2.753838,32,297.94465,1.369509e+14,0.04,8.112498e+12,17.586027
998,51,164.50680,1.634841,0.039362,11568.5010,116.952150,0.427736,25,367.68276,4.716706e+13,...,71,392.67168,1.164570e+14,3.545243,145,315.50320,2.954213e+14,0.37,3.925326e+13,1.893833
999,11,149.25617,26.150842,0.081776,23516.6380,240.576530,0.467134,8,419.10904,6.398235e+13,...,13,355.75066,9.797732e+13,2.875196,46,469.10444,1.641084e+14,0.09,1.255210e+13,7.681110


In [42]:
def make_query(ra, dec, theta, r200, dist, z_upper, z_lower):
    query = f"""SELECT
   p.objid, p.ra, p.dec, p.l, p.b,
   p.petroMag_u-p.extinction_u AS mag_u,
   p.petroMag_g-p.extinction_g AS mag_g,
   p.petroMag_r-p.extinction_r AS mag_r,
   p.petroMag_i-p.extinction_i AS mag_i,
   p.petroMag_z-p.extinction_z AS mag_z,
   s.z, s.zErr, s.velDisp, s.velDispErr,
   passive.logmass as passive_logmass,
   ROUND(pca.mstellar_median, 2) as pca_logmass,
   ROUND(passive.logmass-passive.minlogmass, 3) as logmass_lowerErr,
   ROUND(passive.maxlogmass-passive.logmass, 3) as logmass_upperErr,
   passive.absMagK,
   pca.vdisp_median as pca_vdisp, pca.vdisp_err as pca_vdisp_err,
   zoo.p_el_debiased as frac_ellip,
   zoo.p_cs_debiased as frac_spiral
FROM Galaxy AS p
   JOIN dbo.fGetNearbyObjEq({ra}, {dec}, {theta}) AS GN ON GN.objid = p.objid
   JOIN SpecObj AS s ON s.bestobjid = p.objid
   LEFT JOIN stellarMassPassivePort AS passive ON passive.specobjid = s.specobjid
   LEFT JOIN stellarMassPCAWiscM11 AS pca ON pca.specobjid = s.specobjid
   LEFT JOIN zooSpec AS zoo ON zoo.specobjid = s.specobjid
WHERE
   s.class = 'GALAXY'
   AND s.z BETWEEN {z_lower} AND {z_upper}"""
    return query

In [46]:
def remove_cachefile(listdir):
    try:
        ld = listdir.remove('.DS_store')
    except:
        ld = listdir
    return ld

def wait_download(path):
    count = len(remove_cachefile(os.listdir(path)))
    while True:
        pre_count = count
        time.sleep(5)
        count = len(remove_cachefile(os.listdir(path)))
        if count > pre_count:
            break
            
def get_csv(driver, query):
    driver.find_element(By.XPATH, '//*[@id="clear"]').click()
    driver.find_element(By.XPATH, '//*[@id="cmd"]').send_keys(query)
    driver.find_element(By.XPATH, '//*[@id="transp"]/form/table/tbody/tr[3]/td/table/tbody/tr/td[2]/table/tbody/tr[1]/td[4]/input').click()
    driver.find_element(By.XPATH, '/html/body/div[4]/form/table/tbody/tr[3]/td/table/tbody/tr/td[1]/table/tbody/tr[6]/td/input').click()
    wait_download('./')

def get_html(driver, query):
    driver.find_element(By.XPATH, '//*[@id="clear"]').click()
    driver.find_element(By.XPATH, '//*[@id="cmd"]').send_keys(query)
    driver.find_element(By.XPATH, '/html/body/div[4]/form/table/tbody/tr[3]/td/table/tbody/tr/td[1]/table/tbody/tr[6]/td/input').click()
    
    time.sleep(np.random.randint(3,5))
    first_tab = driver.window_handles[0]
    driver.switch_to.window(window_name=first_tab)

In [51]:
url = "http://skyserver.sdss.org/dr13/en/tools/search/sql.aspx"
driver = set_chrome_driver()
driver.get(url)
N = len(cls_info)

for idx, cls1 in cls_info.loc[621:, :].iterrows():
    ra = cls1['ra']
    dec = cls1['dec']
    dist = cls1['comv_dist']
    r200 = cls1['r_200']
    v_r = cls1['v_rad']
    sigma = cls1['sigma_200']
    theta = multiple * np.arcsin(r200 / dist) * (180 / np.pi) * 60
    print('Index: {0} & process: {1:.2f}%'.format(idx, 100 * idx/N))
    
    z_upper = redshift(v_r + multiple * sigma)
    z_lower = redshift(v_r - multiple * sigma)
    
    query = make_query(ra, dec, theta, r200, dist, z_upper, z_lower)
    get_csv(driver, query)
    
#     if idx > 0:
#         break

Index: 621 & process: 34.50%
Index: 622 & process: 34.56%
Index: 623 & process: 34.61%
Index: 624 & process: 34.67%
Index: 625 & process: 34.72%
Index: 626 & process: 34.78%
Index: 627 & process: 34.83%
Index: 628 & process: 34.89%
Index: 629 & process: 34.94%
Index: 630 & process: 35.00%
Index: 631 & process: 35.06%
Index: 632 & process: 35.11%
Index: 633 & process: 35.17%
Index: 634 & process: 35.22%
Index: 635 & process: 35.28%
Index: 636 & process: 35.33%
Index: 637 & process: 35.39%
Index: 638 & process: 35.44%
Index: 639 & process: 35.50%
Index: 640 & process: 35.56%
Index: 641 & process: 35.61%
Index: 642 & process: 35.67%
Index: 643 & process: 35.72%
Index: 644 & process: 35.78%
Index: 645 & process: 35.83%
Index: 646 & process: 35.89%
Index: 647 & process: 35.94%
Index: 648 & process: 36.00%
Index: 649 & process: 36.06%
Index: 650 & process: 36.11%
Index: 651 & process: 36.17%
Index: 652 & process: 36.22%
Index: 653 & process: 36.28%
Index: 654 & process: 36.33%
Index: 655 & p

Index: 904 & process: 50.22%
Index: 905 & process: 50.28%
Index: 906 & process: 50.33%
Index: 907 & process: 50.39%
Index: 908 & process: 50.44%
Index: 909 & process: 50.50%
Index: 910 & process: 50.56%
Index: 911 & process: 50.61%
Index: 912 & process: 50.67%
Index: 913 & process: 50.72%
Index: 914 & process: 50.78%
Index: 915 & process: 50.83%
Index: 916 & process: 50.89%
Index: 917 & process: 50.94%
Index: 918 & process: 51.00%
Index: 919 & process: 51.06%
Index: 920 & process: 51.11%
Index: 921 & process: 51.17%
Index: 922 & process: 51.22%
Index: 923 & process: 51.28%
Index: 924 & process: 51.33%
Index: 925 & process: 51.39%
Index: 926 & process: 51.44%
Index: 927 & process: 51.50%
Index: 928 & process: 51.56%
Index: 929 & process: 51.61%
Index: 930 & process: 51.67%
Index: 931 & process: 51.72%
Index: 932 & process: 51.78%
Index: 933 & process: 51.83%
Index: 934 & process: 51.89%
Index: 935 & process: 51.94%
Index: 936 & process: 52.00%
Index: 937 & process: 52.06%
Index: 938 & p

Index: 1181 & process: 65.61%
Index: 1182 & process: 65.67%
Index: 1183 & process: 65.72%
Index: 1184 & process: 65.78%
Index: 1185 & process: 65.83%
Index: 1186 & process: 65.89%
Index: 1187 & process: 65.94%
Index: 1188 & process: 66.00%
Index: 1189 & process: 66.06%
Index: 1190 & process: 66.11%
Index: 1191 & process: 66.17%
Index: 1192 & process: 66.22%
Index: 1193 & process: 66.28%
Index: 1194 & process: 66.33%
Index: 1195 & process: 66.39%
Index: 1196 & process: 66.44%
Index: 1197 & process: 66.50%
Index: 1198 & process: 66.56%
Index: 1199 & process: 66.61%
Index: 1200 & process: 66.67%
Index: 1201 & process: 66.72%
Index: 1202 & process: 66.78%
Index: 1203 & process: 66.83%
Index: 1204 & process: 66.89%
Index: 1205 & process: 66.94%
Index: 1206 & process: 67.00%
Index: 1207 & process: 67.06%
Index: 1208 & process: 67.11%
Index: 1209 & process: 67.17%
Index: 1210 & process: 67.22%
Index: 1211 & process: 67.28%
Index: 1212 & process: 67.33%
Index: 1213 & process: 67.39%
Index: 121

Index: 1455 & process: 80.83%
Index: 1456 & process: 80.89%
Index: 1457 & process: 80.94%
Index: 1458 & process: 81.00%
Index: 1459 & process: 81.06%
Index: 1460 & process: 81.11%
Index: 1461 & process: 81.17%
Index: 1462 & process: 81.22%
Index: 1463 & process: 81.28%
Index: 1464 & process: 81.33%
Index: 1465 & process: 81.39%
Index: 1466 & process: 81.44%
Index: 1467 & process: 81.50%
Index: 1468 & process: 81.56%
Index: 1469 & process: 81.61%
Index: 1470 & process: 81.67%
Index: 1471 & process: 81.72%
Index: 1472 & process: 81.78%
Index: 1473 & process: 81.83%
Index: 1474 & process: 81.89%
Index: 1475 & process: 81.94%
Index: 1476 & process: 82.00%
Index: 1477 & process: 82.06%
Index: 1478 & process: 82.11%
Index: 1479 & process: 82.17%
Index: 1480 & process: 82.22%
Index: 1481 & process: 82.28%
Index: 1482 & process: 82.33%
Index: 1483 & process: 82.39%
Index: 1484 & process: 82.44%
Index: 1485 & process: 82.50%
Index: 1486 & process: 82.56%
Index: 1487 & process: 82.61%
Index: 148

Index: 1729 & process: 96.06%
Index: 1730 & process: 96.11%
Index: 1731 & process: 96.17%
Index: 1732 & process: 96.22%
Index: 1733 & process: 96.28%
Index: 1734 & process: 96.33%
Index: 1735 & process: 96.39%
Index: 1736 & process: 96.44%
Index: 1737 & process: 96.50%
Index: 1738 & process: 96.56%
Index: 1739 & process: 96.61%
Index: 1740 & process: 96.67%
Index: 1741 & process: 96.72%
Index: 1742 & process: 96.78%
Index: 1743 & process: 96.83%
Index: 1744 & process: 96.89%
Index: 1745 & process: 96.94%
Index: 1746 & process: 97.00%
Index: 1747 & process: 97.06%
Index: 1748 & process: 97.11%
Index: 1749 & process: 97.17%
Index: 1750 & process: 97.22%
Index: 1751 & process: 97.28%
Index: 1752 & process: 97.33%
Index: 1753 & process: 97.39%
Index: 1754 & process: 97.44%
Index: 1755 & process: 97.50%
Index: 1756 & process: 97.56%
Index: 1757 & process: 97.61%
Index: 1758 & process: 97.67%
Index: 1759 & process: 97.72%
Index: 1760 & process: 97.78%
Index: 1761 & process: 97.83%
Index: 176

In [19]:
def remove_eng(s):
    res = ''
    for char in s:
        if char.isdigit() or char == '_':
            res += char
    return res

def time_extract(f: str) -> str:
    day = remove_eng('_'.join(f.split(' ')[0].split('_')[1:]))
    time = f.split(' ')[1] + ' ' + f.split(' ')[-1].split('.')[0]
    return day + ' ' + time

def check_matching(data, center, err):
    if abs(center[0]-data['ra'].mean()) >= err:
        return True
    elif abs(center[1]-data['dec'].mean()) >= err:
        return True
    else:
        return False

file_ls = os.listdir(data_path)
time_ls = []

for filename in file_ls:
    try:
        t = time_extract(filename)
    except:
        raise Exception(filename)
    time_ls.append(t)

file_df = pd.DataFrame({
    'filename': file_ls,
    'time': time_ls,
})

file_df.time = pd.to_datetime(file_df.time, format='%m_%d_%Y %I_%M_%S %p')
file_df = file_df.sort_values(by='time')
file_df = file_df.reset_index(drop=True)
file_df.index = file_df.index + 1

gal_df_ls = []

for idx, row in file_df.iterrows():
    df = pd.read_csv(data_path + row['filename'], header=1)
    center = (cls_info.loc[idx+1000, 'ra'], cls_info.loc[idx+1000, 'dec'])    
    gal_df_ls.append(df)
    if check_matching(df, center, 0.7):
        print(idx)
        print(df.ra.mean(), df.dec.mean())
        print(center)
        break
    gal_df_ls.append(df)

202
174.791019940972 55.6667351395397
(192.28152, -1.7848679)


In [57]:
def get_missing_data(driver, cls_info, index):
    cls1 = cls_info.loc[index, :]
    ra = cls1['ra']
    dec = cls1['dec']
    dist = cls1['comv_dist']
    r200 = cls1['r_200']
    v_r = cls1['v_rad']
    sigma = cls1['sigma_200']
    theta = multiple * np.arcsin(r200 / dist) * (180 / np.pi) * 60
    print('Index: {0} & process: {1:.2f}%'.format(idx, 100 * idx/N))
    
    z_upper = redshift(v_r + multiple * sigma)
    z_lower = redshift(v_r - multiple * sigma)
    
    query = make_query(ra, dec, theta, r200, dist, z_upper, z_lower)
    get_csv(driver, query)

In [58]:
get_missing_data(driver, cls_info, 1703)

Index: 1800 & process: 100.00%
